In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', 250)

In [3]:
API_KEY = ''

# Ranker Stats 추출

## Ranker Name 추출

In [4]:
#api 호출로 랭커의 기본정보 불러오기
#기본정보에서 랭커의 캐릭터 이름만 추출
headers = {
    'accept': 'application/json',
    'x-nxopen-api-key': API_KEY,
}

params = {
    'date': '2025-05-01',
    'world_name': '스카니아',
}

response_name = requests.get('https://open.api.nexon.com/maplestory/v1/ranking/overall', params=params, headers=headers)

In [5]:
ranker_name_json = response_name.json()
print(ranker_name_json['ranking'][0])

{'date': '2025-05-01', 'world_name': '스카니아', 'ranking': 1, 'character_name': '단솜', 'character_level': 298, 'character_exp': 1121970583144853, 'class_name': '기사단', 'sub_class_name': '윈드브레이커', 'character_popularity': 3441, 'character_guildname': '봄연'}


In [6]:
#랭커 캐릭터들의 이름만 추출하여 리스트 작성
Ranker_name_list = []
for data in ranker_name_json['ranking']:
    Ranker_name_list.append(data['character_name'])
Ranker_name_list

['단솜',
 '카사',
 '서인',
 'Baddy',
 '한계',
 '은사',
 '맛살',
 '가람',
 '알샤',
 '영돌',
 '채콩',
 '누라리한',
 '초기',
 '냥이롬',
 '캅쑤',
 '스포츠',
 '연님',
 '밍안',
 '교서',
 '섀마',
 '딜동차',
 '카키',
 '쏘쿨',
 '사선추격',
 '이어징',
 '케롤',
 '미헤',
 '소주',
 '하늘과땅차이',
 '카시',
 '숙원',
 '추돌',
 '칼별이삼',
 '욱브',
 '끄나',
 '챌리',
 '종교',
 '네코',
 '윤살아',
 '의첨지',
 '남한',
 '하멜른쥐잡이',
 '우정',
 '탕빠',
 '혼나면',
 '복덩',
 '메세지',
 '거대똥파리',
 '톰스',
 '붓싼',
 '때구',
 '퐁뿡',
 '뻬삐뾰',
 '부장',
 '완치',
 '라멜',
 '난또삐',
 '학폭',
 '섈업',
 '컨킹',
 '체저',
 '발락',
 '슈킹',
 '수영',
 '쭈푸름',
 '개강',
 '비류',
 '토푭',
 '중기',
 '모니카',
 '치명',
 '치이',
 '나넹',
 '겨울',
 '봉노',
 '메인',
 '와타코코',
 '매력만빵',
 'Feta',
 '사마목',
 '하와',
 '바통',
 '긴장',
 '저더',
 '민법',
 '매실',
 '사랑의모양',
 '원장',
 '인연님이에요',
 '으꾸',
 '시커만던져요',
 '경보',
 '크공',
 '통그릴',
 '커지',
 '정무',
 '세븐',
 '소떡',
 '후니',
 '랑헤헷',
 '친츄',
 '자너',
 '쏭콜',
 '황후',
 '김도희',
 '아이도루',
 '토끼신',
 '정식',
 '큐리어스',
 '찐돌쿤',
 '드랭',
 '무쥬',
 '성기사',
 '후앵',
 'Kanye',
 '우빙',
 '림픽',
 '영문이름',
 '짠켜',
 '태규',
 '소방청장',
 '여제이',
 '배황',
 '곤봉',
 '워녕',
 '은칭',
 '싼타페7477',
 'Aran2최고다',
 '쪽빛궁모',
 '켠핑',
 '클탐',
 '

In [7]:
Ranker_name_list_1 = Ranker_name_list[:157]
Ranker_name_list_2 = Ranker_name_list[157:]

In [8]:
print(len(Ranker_name_list_1), len(Ranker_name_list_2))

157 43


## Ranker Name을 ocid로 변환

In [9]:
def name_to_ocid(name_list):
    ocid_list = []

    for name in name_list:
        headers = {
        'accept': 'application/json',
        'x-nxopen-api-key': API_KEY,
        }

        params = {
        'character_name': name,
        }

        response_ocid = requests.get('https://open.api.nexon.com/maplestory/v1/id', params=params, headers=headers)
        ocid_list.append(response_ocid.json().values())
    return ocid_list

In [16]:
ocid_list_1 = name_to_ocid(Ranker_name_list_1)

In [10]:
ocid_list_2 = name_to_ocid(Ranker_name_list_2)

## ocid를 이용하여 Character의 Stats 불러오기

In [11]:
#for문은 내부에 넣지 않고 외부에 사용하기
#코드 재사용하기 용이

def Load_Stats(ocid):
    
    headers = {
        'accept': 'application/json',
        'x-nxopen-api-key': API_KEY,
    }

    params = {
        'ocid': ocid,
    }

    response_basic = requests.get('https://open.api.nexon.com/maplestory/v1/character/basic', params=params, headers=headers)
    response_main_stats = requests.get('https://open.api.nexon.com/maplestory/v1/character/stat', params=params, headers=headers)
    response_hyper_stats = requests.get('https://open.api.nexon.com/maplestory/v1/character/hyper-stat', params=params, headers=headers)
    response_item_equipment = requests.get('https://open.api.nexon.com/maplestory/v1/character/item-equipment', params=params, headers=headers)
    response_symbol_equipment = requests.get('https://open.api.nexon.com/maplestory/v1/character/symbol-equipment', params=params, headers=headers)
    
    basic_stats = response_basic.json()
    main_stats = response_main_stats.json()
    hyper_stats = response_hyper_stats.json()
    item_equipment = response_item_equipment.json()
    symbol_equipment = response_symbol_equipment.json()
    
    row0 = {
            'character_name': basic_stats['character_name'],
            'character_gender': basic_stats['character_gender'],
            'character_class': basic_stats['character_class'],
            'character_level': basic_stats['character_level'],
           }
    
    list_main_stats = main_stats['final_stat']
    row1 = {data['stat_name']: data['stat_value'] for data in list_main_stats}
    row1['character_class'] = main_stats['character_class']
    
    list_hyper_stats = hyper_stats['hyper_stat_preset_1']
    row2 = {'Hyper ' + data['stat_type']: data['stat_level'] for data in list_hyper_stats}
    
    list_item_equipment = item_equipment['item_equipment']
    row3 ={}
    for data in list_item_equipment:
        row3.update({'Item_Equipment ' + data['item_equipment_slot']: data['item_name'],
                     'Item_Equipment ' + data['item_equipment_slot'] + ' Starforce': data['starforce']})
        
    list_symbol_equipment = symbol_equipment['symbol']
    row4 = {data['symbol_name']: data['symbol_level'] for data in list_symbol_equipment}
    
    
    row0.update(row1)
    row0.update(row2)
    row0.update(row3)
    row0.update(row4)

    return row0

In [24]:
#api 호출 횟수 제한으로 나눠서 호출
Character_stats_list_1 = []

for ocid in ocid_list_1:
    Character_stats_list_1.append(Load_Stats(ocid))

In [12]:
Character_stats_list_2 = []

for ocid in ocid_list_2:
    Character_stats_list_2.append(Load_Stats(ocid))

In [25]:
len(Character_stats_list_1)

157

In [13]:
len(Character_stats_list_2)

43

In [26]:
df = pd.DataFrame(Character_stats_list_1)

In [14]:
df1 = pd.DataFrame(Character_stats_list_2)

In [15]:
#임시 저장
df.to_csv('maple_data1.csv')

In [16]:
#임시 저장
df1.to_csv('maple_data2.csv')

In [18]:
df1 = pd.read_csv('./maple_data1.csv', index_col = 0)
df2 = pd.read_csv('./maple_data2.csv', index_col = 0)

In [19]:
#나눠진 데이터 합치기
df = pd.concat([df1, df2], ignore_index = True)

In [20]:
df.shape

(200, 126)

## 데이터 전처리

In [21]:
#null 확인
df[df.isnull().any(axis=1)]

,character_name,character_gender,character_class,character_level,최소 스탯공격력,최대 스탯공격력,데미지,보스 몬스터 데미지,최종 데미지,방어율 무시,크리티컬 확률,크리티컬 데미지,상태이상 내성,스탠스,방어력,이동속도,점프력,스타포스,아케인포스,어센틱포스,STR,DEX,INT,LUK,HP,MP,AP 배분 STR,AP 배분 DEX,AP 배분 INT,AP 배분 LUK,AP 배분 HP,AP 배분 MP,아이템 드롭률,메소 획득량,버프 지속시간,공격 속도,일반 몬스터 데미지,재사용 대기시간 감소 (초),재사용 대기시간 감소 (%),재사용 대기시간 미적용,속성 내성 무시,상태이상 추가 데미지,무기 숙련도,추가 경험치 획득,공격력,마력,전투력,소환수 지속시간 증가,Hyper STR,Hyper DEX,Hyper INT,Hyper LUK,Hyper HP,Hyper MP,Hyper DF/TF/PP,Hyper 크리티컬 확률,Hyper 크리티컬 데미지,Hyper 방어율 무시,Hyper 데미지,Hyper 보스 몬스터 공격 시 데미지 증가,Hyper 상태 이상 내성,Hyper 공격력/마력,Hyper 획득 경험치,Hyper 아케인포스,Hyper 일반 몬스터 공격 시 데미지 증가,Item_Equipment 모자,Item_Equipment 모자 Starforce,Item_Equipment 얼굴장식,Item_Equipment 얼굴장식 Starforce,Item_Equipment 눈장식,Item_Equipment 눈장식 Starforce,Item_Equipment 귀고리,Item_Equipment 귀고리 Starforce,Item_Equipment 상의,Item_Equipment 상의 Starforce,Item_Equipment 하의,Item_Equipment 하의 Starforce,Item_Equipment 신발,Item_Equipment 신발 Starforce,Item_Equipment 장갑,Item_Equipment 장갑 Starforce,Item_Equipment 망토,Item_Equipment 망토 Starforce,Item_Equipment 보조무기,Item_Equipment 보조무기 Starforce,Item_Equipment 무기,Item_Equipment 무기 Starforce,Item_Equipment 반지1,Item_Equipment 반지1 Starforce,Item_Equipment 반지2,Item_Equipment 반지2 Starforce,Item_Equipment 반지3,Item_Equipment 반지3 Starforce,Item_Equipment 반지4,Item_Equipment 반지4 Starforce,Item_Equipment 펜던트,Item_Equipment 펜던트 Starforce,Item_Equipment 훈장,Item_Equipment 훈장 Starforce,Item_Equipment 벨트,Item_Equipment 벨트 Starforce,Item_Equipment 어깨장식,Item_Equipment 어깨장식 Starforce,Item_Equipment 포켓 아이템,Item_Equipment 포켓 아이템 Starforce,Item_Equipment 기계 심장,Item_Equipment 기계 심장 Starforce,Item_Equipment 뱃지,Item_Equipment 뱃지 Starforce,Item_Equipment 엠블렘,Item_Equipment 엠블렘 Starforce,Item_Equipment 펜던트2,Item_Equipment 펜던트2 Starforce,아케인심볼 : 레헬른,아케인심볼 : 아르카나,아케인심볼 : 소멸의 여로,아케인심볼 : 모라스,아케인심볼 : 츄츄 아일랜드,아케인심볼 : 에스페라,어센틱심볼 : 세르니움,어센틱심볼 : 아르크스,어센틱심볼 : 오디움,어센틱심볼 : 도원경,어센틱심볼 : 아르테리아,어센틱심볼 : 카르시온,그랜드 어센틱심볼 : 탈라하트
11,누라리한,기타,제로,296,113323801,125915332,191.0,402.0,67.80,87.11,104,77.30,82,100,99999,138,123,283,1350,720,62271,6476,3930,3515,80349,NaN,1498,4,4,4,-270,0,286,99,55,2,125.0,5,6,7,5.0,6.0,90,634.0,6772,1388,166669466,0,7,2,0,0,0,0,7,4,10,0,10,2,0,8,15,0,15,하이네스 워리어헬름,22.0,샤이니 레드 워리어 마이스터 심볼,20.0,미카엘라의 새 안경,5.0,하프 이어링,5.0,이글아이 워리어아머,22.0,트릭스터 워리어팬츠,22.0,앱솔랩스 나이트슈즈,22.0,앱솔랩스 나이트글러브,22.0,앱솔랩스 나이트케이프,22.0,제네시스 라피스,22.0,제네시스 라즐리,22,어웨이크 링,0.0,글로리온 링 : 슈프림,0.0,실버블라썸 링,10.0,마이스터링,17.0,그리드 펜던트,5.0,빅토리아컵,0.0,몽환의 벨트,22.0,앱솔랩스 나이트숄더,22.0,저주받은 적의 마도서,0,페어리 하트,8.0,창세의 뱃지,0,이터널 타임 엠블렘,0,준비된 정령의 펜던트,0.0,20,20,20,20,20,20,11,11,11,11,11,11,6
16,연님,여,아델,296,64033017,70365951,104.0,286.0,68.60,72.98,130,86.50,71,100,99999,160,123,262,1350,750,56851,5746,2697,3056,79334,24556.0,1498,4,4,4,0,0,264,150,55,6,145.0,4,6,7,5.0,0.0,91,396.0,6750,1246,118726952,0,6,2,0,0,0,0,0,6,14,11,13,14,0,7,0,0,0,하이네스 워리어헬름,22.0,레드 워리어 마이스터 심볼,8.0,미카엘라의 새 안경,5.0,하프 이어링,5.0,이글아이 워리어아머,22.0,트릭스터 워리어팬츠,22.0,앱솔랩스 나이트슈즈,22.0,앱솔랩스 나이트글러브,22.0,앱솔랩스 나이트케이프,22.0,노블 브레이슬릿,0.0,제네시스 튜너,22,SS급 마스터 쥬얼링,0.0,마이스터 올마이티링,5.0,버서커의 마이스터 임모탈 링,5.0,마이스터링,17.0,베어스 그린 펜던트,8.0,NaN,NaN,타일런트 히아데스 벨트,10.0,앱솔랩스 나이트숄더,22.0,저주받은 적의 마도서,0,페어리 하트,8.0,크리스탈 웬투스 뱃지,0,골드 나이트 엠블렘,0,준비된 정령의 펜던트,0.0,20,20,20,20,20,20,11,11,11,11,11,11,9
18,교서,여,신궁,296,1374644,9164290,71.0,187.0,10.00,62.50,60,57.60,30,100,17011,129,110,22,1320,760,1903,32190,728,1016,23806,12744.0,4,1498,4,4,0,0,41,51,56,4,94.0,0,6,3,5.0,0.0,15,205.0,2762,337,23716372,20,5,8,0,0,0,0,0,0,12,0,12,0,0,8,15,0,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제네시스 크로스보우,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,그리드 펜던트,0.0,칠요의 몬스터파커,0.0,NaN,NaN,NaN,NaN,저주받은 녹의 마도서,0,NaN,NaN,창세의 뱃지,0,미트라의 분노 : 궁수,0,NaN,NaN,20,20,20,20,20,20,11,11,11,11,11,11,10
28,하늘과땅차이,여,다크나이트,295,40020643,44467378,102.0,301.0,122.75,85.76,97,122.95,65,100,74737,160,123,60,1350,750,43035,4213,2781,2787,83407,38168.0,1493,4,4,4,-412,-240,80,50,60,4,107.0,0,6,7,5.0,0.0,90,171.0,3761,742,48562179,0,5,1,0,0,0,0,0,3,14,11,13,15,6,0,0,0,0,하이네스 워

In [22]:
len(df[df.isnull().any(axis=1)])

26

In [23]:
#column을 type 별로 분류
numerical_list = []
categorical_list = []
for col_name in df.columns:
    try:
        if int(df.iloc[0][col_name]):
            numerical_list.append(col_name)
    except:
        categorical_list.append(col_name)

In [24]:
print(numerical_list)

['character_level', '최소 스탯공격력', '최대 스탯공격력', '데미지', '보스 몬스터 데미지', '최종 데미지', '방어율 무시', '크리티컬 확률', '크리티컬 데미지', '상태이상 내성', '스탠스', '방어력', '이동속도', '점프력', '스타포스', '아케인포스', '어센틱포스', 'STR', 'DEX', 'INT', 'LUK', 'HP', 'MP', 'AP 배분 STR', 'AP 배분 DEX', 'AP 배분 INT', 'AP 배분 LUK', 'AP 배분 HP', 'AP 배분 MP', '아이템 드롭률', '메소 획득량', '버프 지속시간', '공격 속도', '일반 몬스터 데미지', '재사용 대기시간 감소 (초)', '재사용 대기시간 감소 (%)', '재사용 대기시간 미적용', '속성 내성 무시', '무기 숙련도', '추가 경험치 획득', '공격력', '마력', '전투력', '소환수 지속시간 증가', 'Hyper STR', 'Hyper DEX', 'Hyper 크리티컬 확률', 'Hyper 크리티컬 데미지', 'Hyper 방어율 무시', 'Hyper 데미지', 'Hyper 보스 몬스터 공격 시 데미지 증가', 'Hyper 공격력/마력', 'Item_Equipment 모자 Starforce', 'Item_Equipment 얼굴장식 Starforce', 'Item_Equipment 눈장식 Starforce', 'Item_Equipment 귀고리 Starforce', 'Item_Equipment 상의 Starforce', 'Item_Equipment 하의 Starforce', 'Item_Equipment 신발 Starforce', 'Item_Equipment 장갑 Starforce', 'Item_Equipment 망토 Starforce', 'Item_Equipment 무기 Starforce', 'Item_Equipment 반지1 Starforce', 'Item_Equipment 반지2 Starforce', 'Item_Equipment 반지3

In [25]:
print(categorical_list)

['character_name', 'character_gender', 'character_class', 'Item_Equipment 모자', 'Item_Equipment 얼굴장식', 'Item_Equipment 눈장식', 'Item_Equipment 귀고리', 'Item_Equipment 상의', 'Item_Equipment 하의', 'Item_Equipment 신발', 'Item_Equipment 장갑', 'Item_Equipment 망토', 'Item_Equipment 보조무기', 'Item_Equipment 무기', 'Item_Equipment 반지1', 'Item_Equipment 반지2', 'Item_Equipment 반지3', 'Item_Equipment 반지4', 'Item_Equipment 펜던트', 'Item_Equipment 훈장', 'Item_Equipment 벨트', 'Item_Equipment 어깨장식', 'Item_Equipment 포켓 아이템', 'Item_Equipment 기계 심장', 'Item_Equipment 뱃지', 'Item_Equipment 엠블렘', 'Item_Equipment 펜던트2']


In [26]:
df = df.reset_index()


In [27]:
df_dropna = df.dropna()
df_dropna[df_dropna.notnull().any(axis=1)]

,index,character_name,character_gender,character_class,character_level,최소 스탯공격력,최대 스탯공격력,데미지,보스 몬스터 데미지,최종 데미지,방어율 무시,크리티컬 확률,크리티컬 데미지,상태이상 내성,스탠스,방어력,이동속도,점프력,스타포스,아케인포스,어센틱포스,STR,DEX,INT,LUK,HP,MP,AP 배분 STR,AP 배분 DEX,AP 배분 INT,AP 배분 LUK,AP 배분 HP,AP 배분 MP,아이템 드롭률,메소 획득량,버프 지속시간,공격 속도,일반 몬스터 데미지,재사용 대기시간 감소 (초),재사용 대기시간 감소 (%),재사용 대기시간 미적용,속성 내성 무시,상태이상 추가 데미지,무기 숙련도,추가 경험치 획득,공격력,마력,전투력,소환수 지속시간 증가,Hyper STR,Hyper DEX,Hyper INT,Hyper LUK,Hyper HP,Hyper MP,Hyper DF/TF/PP,Hyper 크리티컬 확률,Hyper 크리티컬 데미지,Hyper 방어율 무시,Hyper 데미지,Hyper 보스 몬스터 공격 시 데미지 증가,Hyper 상태 이상 내성,Hyper 공격력/마력,Hyper 획득 경험치,Hyper 아케인포스,Hyper 일반 몬스터 공격 시 데미지 증가,Item_Equipment 모자,Item_Equipment 모자 Starforce,Item_Equipment 얼굴장식,Item_Equipment 얼굴장식 Starforce,Item_Equipment 눈장식,Item_Equipment 눈장식 Starforce,Item_Equipment 귀고리,Item_Equipment 귀고리 Starforce,Item_Equipment 상의,Item_Equipment 상의 Starforce,Item_Equipment 하의,Item_Equipment 하의 Starforce,Item_Equipment 신발,Item_Equipment 신발 Starforce,Item_Equipment 장갑,Item_Equipment 장갑 Starforce,Item_Equipment 망토,Item_Equipment 망토 Starforce,Item_Equipment 보조무기,Item_Equipment 보조무기 Starforce,Item_Equipment 무기,Item_Equipment 무기 Starforce,Item_Equipment 반지1,Item_Equipment 반지1 Starforce,Item_Equipment 반지2,Item_Equipment 반지2 Starforce,Item_Equipment 반지3,Item_Equipment 반지3 Starforce,Item_Equipment 반지4,Item_Equipment 반지4 Starforce,Item_Equipment 펜던트,Item_Equipment 펜던트 Starforce,Item_Equipment 훈장,Item_Equipment 훈장 Starforce,Item_Equipment 벨트,Item_Equipment 벨트 Starforce,Item_Equipment 어깨장식,Item_Equipment 어깨장식 Starforce,Item_Equipment 포켓 아이템,Item_Equipment 포켓 아이템 Starforce,Item_Equipment 기계 심장,Item_Equipment 기계 심장 Starforce,Item_Equipment 뱃지,Item_Equipment 뱃지 Starforce,Item_Equipment 엠블렘,Item_Equipment 엠블렘 Starforce,Item_Equipment 펜던트2,Item_Equipment 펜던트2 Starforce,아케인심볼 : 레헬른,아케인심볼 : 아르카나,아케인심볼 : 소멸의 여로,아케인심볼 : 모라스,아케인심볼 : 츄츄 아일랜드,아케인심볼 : 에스페라,어센틱심볼 : 세르니움,어센틱심볼 : 아르크스,어센틱심볼 : 오디움,어센틱심볼 : 도원경,어센틱심볼 : 아르테리아,어센틱심볼 : 카르시온,그랜드 어센틱심볼 : 탈라하트
0,0,단솜,여,윈드브레이커,299,101871166,119848429,113.0,249.0,78.05,85.95,84,89.00,70,100,66952,160,123,344,1350,760,8524,64505,4568,4245,47251,23051.0,4,1513,4,4,-391,-198,241,151,53,4,141.0,5,6,6,5.0,0.0,85,315.0,9120,1920,159188587,10,2,2,0,0,0,0,0,10,12,14,12,14,0,7,0,0,0,하이네스 레인져베레,22.0,샤이니 레드 아처 마이스터 심볼,20.0,마력이 깃든 안대,22.0,오션 글로우 이어링,22.0,이글아이 레인져후드,22.0,트릭스터 레인져팬츠,22.0,아케인셰이드 아처슈즈,22.0,아케인셰이드 아처글러브,22.0,아케인셰이드 아처케이프,22.0,에레브의 광휘,0.0,제네시스 보우,22,마이스터 올마이티링,5.0,가디언의 마이스터 임모탈 링,5.0,버서커의 마이스터 임모탈 링,5.0,마이스터링,22.0,도미네이터 펜던트,22.0,칠요의 몬스터파커,0.0,몽환의 벨트,22.0,아케인셰이드 아처숄더,22.0,저주받은 적의 마도서,0,티타늄 하트,8.0,칠요의 뱃지,0,미트라의 분노 : 궁수,0,준비된 정령의 펜던트,0.0,20,20,20,20,20,20,11,11,11,11,11,11,10
1,1,카사,남,팬텀,298,171273841,190304265,125.0,337.0,126.51,93.88,96,109.45,92,100,66345,160,123,370,1350,770,8462,12053,7207,77561,73877,36789.0,4,4,4,1508,-482,-294,271,147,95,5,137.0,5,6,7,5.0,6.0,90,349.0,8912,2360,301674086,12,1,2,0,6,0,0,0,0,14,11,13,15,5,6,0,0,0,에테르넬 시프반다나,22.0,샤이니 레드 시프 마이스터 심볼,20.0,마력이 깃든 안대,22.0,오션 글로우 이어링,22.0,에테르넬 시프셔츠,22.0,에테르넬 시프팬츠,22.0,앱솔랩스 시프슈즈,22.0,앱솔랩스 시프글러브,22.0,앱솔랩스 시프케이프,22.0,데르니에 카르트,0.0,제네시스 케인,22,마이스터 올마이티링,5.0,마이스터링,22.0,여명의 가디언 엔젤 링,22.0,글로리온 링 : 슈프림,0.0,데이브레이크 펜던트,22.0,THE RED,0.0,몽환의 벨트,22.0,에테르넬 시프숄더,22.0,저주받은 황의 마도서,0,컴플리트 언더컨트롤,22.0,창세의 뱃지,0,미트라의 분노 : 도적,0,준비된 정령의 펜던트,0.0,20,20,20,20,20,20,11,11,11,11,11,11,11
2,2,서인,남,듀얼블레이더,297,184914709,203202975,136.0,394.0,117.07,84.92,101,146.55,59,100,95392,160,123,353,1350,760,8653,11538,7613,72520,74567,44107.0,4,4,4,1503,0,0,290,177,103,6,144.0,5,6,0,5.0,6.0,91,680.0,9834,2060,360413700,30,2,2,1,5,0,0,0,3,14,9,14,15,0,7,0,0,0,하이네스 어새신보닛,22.0,샤이니 레드 시프 마이스터 심볼,20.0,미카엘라의 새 안경,5.0,오션 글로우 이어링,22.0,에테르넬 시프셔츠,22.0,에테르넬 시프팬츠,22.0,아케인셰이드 시프슈즈,22.0,아케인셰이드 시프글러브,22.0,아케인셰이드 시프케이프,22.0,아케인셰이드 블레이드,23.0,제네시스 대거,22,마이스터 올마이티링,5.0,마이스터링,22.0,글로리온 링 : 슈프림,0.0,테네브리스 원정대 반지,0.0,도미네이터 펜던트,22.0,카오스 벨룸 킬러,0.0,몽환의 벨트,22.0,아케인셰이드 시프숄더,22.0,저주받은 황의 마도서,0,컴플리트 언더컨트롤,21.0,창세의 뱃지,0,골드 메이플리프 엠블렘,0,준비된 정령의 펜던트,0.0,20,20,20,20,20,20,11,11,11,11,11,11,10
3,3,Baddy,남,메카닉,297,121366622,14112397

In [ ]:
#null제거 후 포맷을 int로 변경
for col_name in numerical_list:
    df_dropna[col_name] = df_dropna[col_name].apply(lambda x:  int(x))

In [29]:
df_dropna.info(True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 199
Data columns (total 127 columns):
 #    Column                           Dtype  
---   ------                           -----  
 0    index                            int64  
 1    character_name                   object 
 2    character_gender                 object 
 3    character_class                  object 
 4    character_level                  int64  
 5    최소 스탯공격력                         int64  
 6    최대 스탯공격력                         int64  
 7    데미지                              int64  
 8    보스 몬스터 데미지                       int64  
 9    최종 데미지                           int64  
 10   방어율 무시                           int64  
 11   크리티컬 확률                          int64  
 12   크리티컬 데미지                         int64  
 13   상태이상 내성                          int64  
 14   스탠스                              int64  
 15   방어력                              int64  
 16   이동속도                             int64  
 

In [30]:
print("중복된 항목 수: ", len(df_dropna[df_dropna.duplicated()]))

중복된 항목 수:  0


In [31]:
#전처리된 데이터 저장
df_dropna.to_csv('maple_dropna.csv')

# 데이터 불러오기

In [32]:
df = pd.read_csv('./maple_dropna.csv')

In [33]:
df.head()

,Unnamed: 0,index,character_name,character_gender,character_class,character_level,최소 스탯공격력,최대 스탯공격력,데미지,보스 몬스터 데미지,최종 데미지,방어율 무시,크리티컬 확률,크리티컬 데미지,상태이상 내성,스탠스,방어력,이동속도,점프력,스타포스,아케인포스,어센틱포스,STR,DEX,INT,LUK,HP,MP,AP 배분 STR,AP 배분 DEX,AP 배분 INT,AP 배분 LUK,AP 배분 HP,AP 배분 MP,아이템 드롭률,메소 획득량,버프 지속시간,공격 속도,일반 몬스터 데미지,재사용 대기시간 감소 (초),재사용 대기시간 감소 (%),재사용 대기시간 미적용,속성 내성 무시,상태이상 추가 데미지,무기 숙련도,추가 경험치 획득,공격력,마력,전투력,소환수 지속시간 증가,Hyper STR,Hyper DEX,Hyper INT,Hyper LUK,Hyper HP,Hyper MP,Hyper DF/TF/PP,Hyper 크리티컬 확률,Hyper 크리티컬 데미지,Hyper 방어율 무시,Hyper 데미지,Hyper 보스 몬스터 공격 시 데미지 증가,Hyper 상태 이상 내성,Hyper 공격력/마력,Hyper 획득 경험치,Hyper 아케인포스,Hyper 일반 몬스터 공격 시 데미지 증가,Item_Equipment 모자,Item_Equipment 모자 Starforce,Item_Equipment 얼굴장식,Item_Equipment 얼굴장식 Starforce,Item_Equipment 눈장식,Item_Equipment 눈장식 Starforce,Item_Equipment 귀고리,Item_Equipment 귀고리 Starforce,Item_Equipment 상의,Item_Equipment 상의 Starforce,Item_Equipment 하의,Item_Equipment 하의 Starforce,Item_Equipment 신발,Item_Equipment 신발 Starforce,Item_Equipment 장갑,Item_Equipment 장갑 Starforce,Item_Equipment 망토,Item_Equipment 망토 Starforce,Item_Equipment 보조무기,Item_Equipment 보조무기 Starforce,Item_Equipment 무기,Item_Equipment 무기 Starforce,Item_Equipment 반지1,Item_Equipment 반지1 Starforce,Item_Equipment 반지2,Item_Equipment 반지2 Starforce,Item_Equipment 반지3,Item_Equipment 반지3 Starforce,Item_Equipment 반지4,Item_Equipment 반지4 Starforce,Item_Equipment 펜던트,Item_Equipment 펜던트 Starforce,Item_Equipment 훈장,Item_Equipment 훈장 Starforce,Item_Equipment 벨트,Item_Equipment 벨트 Starforce,Item_Equipment 어깨장식,Item_Equipment 어깨장식 Starforce,Item_Equipment 포켓 아이템,Item_Equipment 포켓 아이템 Starforce,Item_Equipment 기계 심장,Item_Equipment 기계 심장 Starforce,Item_Equipment 뱃지,Item_Equipment 뱃지 Starforce,Item_Equipment 엠블렘,Item_Equipment 엠블렘 Starforce,Item_Equipment 펜던트2,Item_Equipment 펜던트2 Starforce,아케인심볼 : 레헬른,아케인심볼 : 아르카나,아케인심볼 : 소멸의 여로,아케인심볼 : 모라스,아케인심볼 : 츄츄 아일랜드,아케인심볼 : 에스페라,어센틱심볼 : 세르니움,어센틱심볼 : 아르크스,어센틱심볼 : 오디움,어센틱심볼 : 도원경,어센틱심볼 : 아르테리아,어센틱심볼 : 카르시온,그랜드 어센틱심볼 : 탈라하트
0,0,0,단솜,여,윈드브레이커,299,101871166,119848429,113,249,78,85,84,89,70,100,66952,160,123,344,1350,760,8524,64505,4568,4245,47251,23051,4,1513,4,4,-391,-198,241,151,53,4,141,5,6,6,5,0.0,85,315,9120,1920,159188587,10,2,2,0,0,0,0,0,10,12,14,12,14,0,7,0,0,0,하이네스 레인져베레,22,샤이니 레드 아처 마이스터 심볼,20,마력이 깃든 안대,22,오션 글로우 이어링,22,이글아이 레인져후드,22,트릭스터 레인져팬츠,22,아케인셰이드 아처슈즈,22,아케인셰이드 아처글러브,22,아케인셰이드 아처케이프,22,에레브의 광휘,0.0,제네시스 보우,22,마이스터 올마이티링,5,가디언의 마이스터 임모탈 링,5,버서커의 마이스터 임모탈 링,5,마이스터링,22,도미네이터 펜던트,22,칠요의 몬스터파커,0.0,몽환의 벨트,22,아케인셰이드 아처숄더,22,저주받은 적의 마도서,0,티타늄 하트,8,칠요의 뱃지,0,미트라의 분노 : 궁수,0,준비된 정령의 펜던트,0.0,20,20,20,20,20,20,11,11,11,11,11,11,10
1,1,1,카사,남,팬텀,298,171273841,190304265,125,337,126,93,96,109,92,100,66345,160,123,370,1350,770,8462,12053,7207,77561,73877,36789,4,4,4,1508,-482,-294,271,147,95,5,137,5,6,7,5,6.0,90,349,8912,2360,301674086,12,1,2,0,6,0,0,0,0,14,11,13,15,5,6,0,0,0,에테르넬 시프반다나,22,샤이니 레드 시프 마이스터 심볼,20,마력이 깃든 안대,22,오션 글로우 이어링,22,에테르넬 시프셔츠,22,에테르넬 시프팬츠,22,앱솔랩스 시프슈즈,22,앱솔랩스 시프글러브,22,앱솔랩스 시프케이프,22,데르니에 카르트,0.0,제네시스 케인,22,마이스터 올마이티링,5,마이스터링,22,여명의 가디언 엔젤 링,22,글로리온 링 : 슈프림,0,데이브레이크 펜던트,22,THE RED,0.0,몽환의 벨트,22,에테르넬 시프숄더,22,저주받은 황의 마도서,0,컴플리트 언더컨트롤,22,창세의 뱃지,0,미트라의 분노 : 도적,0,준비된 정령의 펜던트,0.0,20,20,20,20,20,20,11,11,11,11,11,11,11
2,2,2,서인,남,듀얼블레이더,297,184914709,203202975,136,394,117,84,101,146,59,100,95392,160,123,353,1350,760,8653,11538,7613,72520,74567,44107,4,4,4,1503,0,0,290,177,103,6,144,5,6,0,5,6.0,91,680,9834,2060,360413700,30,2,2,1,5,0,0,0,3,14,9,14,15,0,7,0,0,0,하이네스 어새신보닛,22,샤이니 레드 시프 마이스터 심볼,20,미카엘라의 새 안경,5,오션 글로우 이어링,22,에테르넬 시프셔츠,22,에테르넬 시프팬츠,22,아케인셰이드 시프슈즈,22,아케인셰이드 시프글러브,22,아케인셰이드 시프케이프,22,아케인셰이드 블레이드,23.0,제네시스 대거,22,마이스터 올마이티링,5,마이스터링,22,글로리온 링 : 슈프림,0,테네브리스 원정대 반지,0,도미네이터 펜던트,22,카오스 벨룸 킬러,0.0,몽환의 벨트,22,아케인셰이드 시프숄더,22,저주받은 황의 마도서,0,컴플리트 언더컨트롤,21,창세의 뱃지,0,골드 메이플리프 엠블렘,0,준비된 정령의 펜던트,0.0,20,20,20,20,20,20,11,11,11,11,11,11,10
3,3,3,Baddy,남,메카닉,297,121366622,141123976,183,299,56,81,109,121,80,100,53383,160,123,330,1350,740,8194,66537,5267,4546,95955,56595,4,1503,4,4,-303,-270,268,142,53,5,149,5,6,0,5,6.0,86,61